In [1]:
import string
import math
import random

In [2]:
def clear_message(text, stop_symbols):
    text = text.lower()
    return ''.join([i for i in text if i not in stop_symbols])


def get_random_key():
    str_var = list(alphabet)
    random.shuffle(str_var)
    random_key = ''.join(str_var)
    return random_key


def create_cipher_dict(cipher):
    cipher_dict = {}
    for i in range(len(alphabet)):
        cipher_dict[alphabet[i]] = cipher[i]
    return cipher_dict


def create_ciphertext(text, cipher):
    cipher_dict = create_cipher_dict(cipher)
    text = list(text)
    ciphertext = ''
    for elem in text:
        if elem in cipher_dict:
            ciphertext += cipher_dict[elem]
        else:
            ciphertext += ' '
    return ciphertext


def reverse_cipher(cipher):
    cipher_dict = create_cipher_dict(cipher)
    reverse_dict = {value: key for key, value in cipher_dict.items()}
    reversed_cipher = ''
    for i in alphabet:
        reversed_cipher += reverse_dict[i]
    return reversed_cipher


def compute_frequency_in_corpus(longtext_path):
    duplets_frequency = {}
    try:
        with open(longtext_path) as f:
            for line in f:
                data = list(line.strip())
                for i in range(len(data)-1):
                    alpha_i = data[i].lower()
                    alpha_j = data[i+1].lower()
                    if alpha_i in alphabet and alpha_j in alphabet:
                        key = alpha_i+alpha_j
                        if key in duplets_frequency:
                            duplets_frequency[key] += 1
                        else:
                            duplets_frequency[key] = 1
    except:
        with open(longtext_path, encoding='utf-8') as f:
            for line in f:
                data = list(line.strip())
                for i in range(len(data)-1):
                    alpha_i = data[i].lower()
                    alpha_j = data[i+1].lower()
                    if alpha_i in alphabet and alpha_j in alphabet:
                        key = alpha_i + alpha_j
                        if key in duplets_frequency:
                            duplets_frequency[key] += 1
                        else:
                            duplets_frequency[key] = 1
    return duplets_frequency

In [3]:
class MCMC_decrypt:
    def __init__(self, cipher_text, alphabet, duplets_frequency_in_corpus, iters=10000):
        self.iters = iters
        self.cipher_text = cipher_text
        self.alphabet = alphabet
        self.duplets_frequency_in_corpus = duplets_frequency_in_corpus

    def __create_cipher_dict(self, cipher):
        cipher_dict = {}
        for i in range(len(self.alphabet)):
            cipher_dict[self.alphabet[i]] = cipher[i]
        return cipher_dict

    def create_ciphertext(self, text, cipher):
        cipher_dict = self.__create_cipher_dict(cipher)
        text = list(text)
        ciphertext = ''
        for elem in text:
            if elem in cipher_dict:
                ciphertext += cipher_dict[elem]
            else:
                ciphertext += ' '
        return ciphertext

    def __compute_frequency_in_ciphertext(self, text):
        duplets_frequency = {}
        data = list(text.strip())
        for i in range(len(data)-1):
            alpha_i = data[i]
            alpha_j = data[i+1]
            if alpha_i in self.alphabet and alpha_j in self.alphabet:
                key = alpha_i+alpha_j
            key = alpha_i+alpha_j
            if key in duplets_frequency:
                duplets_frequency[key] += 1
            else:
                duplets_frequency[key] = 1
        return duplets_frequency

    def compute_score(self, text, cipher):
        cipher_dict = self.__create_cipher_dict(cipher)
        decrypted_text = self.create_ciphertext(text, cipher)
        duplets_frequency_in_ciphertext = self.__compute_frequency_in_ciphertext(decrypted_text)
        score = 0
        for duplet, count in duplets_frequency_in_ciphertext.items():
            if duplet in self.duplets_frequency_in_corpus:
                score += count * math.log(self.duplets_frequency_in_corpus[duplet])
        return score

    def __generate_cipher(self, cipher):
        pos1 = random.randint(0, len(list(cipher))-1)
        pos2 = random.randint(0, len(list(cipher))-1)
        if pos1 == pos2:
            return self.__generate_cipher(cipher)
        else:
            cipher = list(cipher)
            cipher[pos1], cipher[pos2] = cipher[pos2], cipher[pos1]
            return ''.join(cipher)

    def __random_coin(self, p):
        unif = random.uniform(0, 1)
        if unif >= p:
            return False
        else:
            return True

    def fit(self):
        current_cipher = self.alphabet
        self.best_state = ''
        score = 0
        for i in range(self.iters):
            proposed_cipher = self.__generate_cipher(current_cipher)
            score_current_cipher = self.compute_score(self.cipher_text, current_cipher)
            score_proposed_cipher = self.compute_score(self.cipher_text, proposed_cipher)
            acceptance_probability = min(1, math.exp(score_proposed_cipher-score_current_cipher))
            if score_current_cipher > score:
                self.best_state = current_cipher
                score = score_current_cipher
            if self.__random_coin(acceptance_probability):
                current_cipher = proposed_cipher
            if i % 1000 == 0:
                print('iter', i, ':', self.create_ciphertext(self.cipher_text, current_cipher)[0:99])

    def predict(self, text=None):
        if text == None:
            return self.create_ciphertext(self.cipher_text, self.best_state)
        else:
            return self.create_ciphertext(text, self.best_state)

    def fit_predict(self, text=None):
        self.fit()
        return self.predict(text)

    def return_best_state(self):
        return self.best_state

    def return_best_score(self):
        return self.compute_score(self.cipher_text, self.best_state)

In [4]:
alphabet = ' абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
duplets_frequency_in_corpus = compute_frequency_in_corpus('corpus.txt')
stop_symbols = string.punctuation + '\n\xa0«»\t—…'

In [5]:
test_message = 'Англичане утверждают, что немецкий народ сопротивляется тотальным военным мерам правительства. Будто он хочет не тотальной войны, а капитуляции. Я спрашиваю вас: хотите ли вы тотальной войны? Хотите ли вы вести ее, даже если надо вести ее еще тотальнее и радикальнее, чем мы сегодня вообще можем себе представить? Фюрер ждет от нас таких свершений, которые затмят все, что было до сих пор. Мы хотим быть на высоте его требований. Мы гордимся им, а он должен иметь возможность гордиться нами <…> Нация готова ко всему. Фюрер приказал, мы следуем за ним. В этот час национального осмысления и внутреннего подъема мы еще вернее и нерушимее будем верить в победу. Мы видим победу в осязаемой близи, нам надо только протянуть руку и схватить ее. Мы должны найти в себе решимость поставить на службу ей абсолютно все. Таково веление времени. А потому наш лозунг: «Вставай, народ, да грядет буря!»'
test_text = clear_message(test_message, stop_symbols)
test_cipher = get_random_key()
test_ciphertext = create_ciphertext(test_text, test_cipher)

In [6]:
mc_test = MCMC_decrypt(test_ciphertext, alphabet, duplets_frequency_in_corpus)
mc_test.fit()
print('\n', 'MCMC decode:', mc_test.return_best_state())
print('\n', 'True decode:', reverse_cipher(test_cipher))
print('\n', 'Decoded Text:', mc_test.predict())

iter 0 : члжоьючлкнхмркёгйчбмнюманлкэксцьшнлчёайнфауёамьропкмфпнмамчо ляэнраклляэнэкёчэнуёчрьмко фмрчнвхйман
iter 1000 : енброяени ктвидглейт ята нисишпом недал уаздатоврюитую татерыньс ваинньс сидес здевотирыутве чклта 
iter 2000 : онзтипоне улмерждойл пла невешгия норад сакралимтюелсю лалотьныв маенныв веров кромилетьслмо чудла 
iter 3000 : онзличоне утверждоют чта немещгий норад сакративляется татольным ваенным мером кровительство пудта 
iter 4000 : анзличане утверждают что немещбий народ сокротивляется тотальным военным мерам кравительства пудто 
iter 5000 : англичане утверждают что немещкий народ сопротивляется тотальным военным мерам правительства будто 
iter 6000 : англичане утверждают что немецкий народ сопротивляется тотальным военным мерам правительства будто 
iter 7000 : англичане утверждают что немецкий народ сопротивляется тотальным военным мерам правительства будто 
iter 8000 : англичане утверждают что немецкий народ сопротивляется тотальным военным мерам правительства бу

In [7]:
cipher = 'мюное шхкьаёъыпивжлтзэбгйщчядрцуфс'
ciphertext = 'щдшэсздбгдтиылъирдвсдпирвясндждягфгдогъзд бгдсынъьисздегясздргсдовяг биъьисзднтдбнёдщдыифвсиадшсвдсвдтиыифислоиадмвжэмиадориъгаджижд бгдмыгъясиорщрвяздбгжвсвывцдявфясогббвясзадндодыгтэрзсисгддвънбдшг въибдмыншг дъвоврзбвдяжыв бвхвдыит гыидолёвънсдщдбнюнцджиждпгдусвдмврэшнрвяз'

In [8]:
mc = MCMC_decrypt(ciphertext, alphabet, duplets_frequency_in_corpus, iters=20000)
mc.fit()
print('\n', 'MCMC decode:', mc.return_best_state())
print('\n', 'True decode:', cipher)
print('\n', 'Decoded Text:', mc.predict())

iter 0 : щдшэсздбгдтиылъирдвсдпирвясндждягфгдогъзд бгдсынъьисздегясздргсдовяг биъьисзднтдбнёдщдыифвсиадшсвдс
iter 1000 : ьдрбведнадз лпк идсвдц истводядтачадуакедмнадвлокж ведшатведиавдустамн кж ведозднойдьдл чсв гдрвсдв
iter 2000 : ждрбводнедю лшк идсвдь иствадядтепедуекодмнедвлакц водъетводиевдустемн кц водаюднайдждл псв гдрвсдв
iter 3000 : йсрбвуснесю лшк дсовсь дотвасястепесиекусмнесвлакч вусъетвусдевсиотемн кч вусаюснахсйсл пов зсрвосв
iter 4000 : г шувы не зарьмал ов жалотви к тече семы дне вримфавы эетвы лев сотеднамфавы из них г рачовая шво в
iter 5000 : я бувы не зарьдал ов чалотви к теше седы мне вриджавы эетвы лев сотемнаджавы из ниц я рашовай бво в
iter 6000 : я бувы не зарьмал ов жалотви к теше семы дне вримчавы фетвы лев сотеднамчавы из ниц я рашовай бво в
iter 7000 : я бувы не зарьдал ов жалотви к тече седы мне вридшавы фетвы лев сотемнадшавы из ниц я рачовай бво в
iter 8000 : я бувы не зарьдал ов жалотви к тече седы мне вридшавы фетвы лев сотемнадшавы из ниц я рачовай б